# reader

In [1]:
import pandas as pd
import numpy as np
import imaplib
import base64
import os
import email
from datetime import timedelta
import datetime

In [2]:
def get_body(msg):
    if msg.is_multipart():
        return get_body(msg.get_payload(0))
    else:
        return msg.get_payload(None, True)

In [3]:
def get_attachments(msg):
    for part in msg.walk():
        if part.get_content_maintype() == "multipart":
            continue
        if part.get("Content-Disposition") is None:
            continue
        fileName = part.get_filename()

        if bool(fileName):
            filePath = os.path.join(attachment_dir, fileName)
            with open(filePath, "wb") as f:
                f.write(part.get_payload(decode=True))

In [4]:
def get_emails(result_bytes):
    msgs = []
    for num in result_bytes[0].split():
        typ, data = con.fetch(num, '(RFC822)')
        msgs.append(data)
    return msgs

---

# read

In [13]:
wave_2 = pd.read_excel("w2_answers_0828.xlsx")#.drop(columns=["Unnamed: 0"])

In [6]:
addresses = list(set(wave_2["email_from"]))

In [7]:
l = []
for i in range(wave_2.shape[0]):
    if wave_2.loc[i, "email_to"].split("@")[1] != "gmail.com":
        l.append(wave_2.loc[i, "email_to"].split("@")[1])

dic = {i: l.count(i) for i in l}

hosts = list(set(l))

hosts_cleaned = []
for i in hosts:
    if dic[i] != 1:
        hosts_cleaned.append(i)
        
hosts_cleaned.append("gmail.com")

In [8]:
not_found = pd.DataFrame(columns=["sender", "msg_body", "account"])

for address in addresses:

    email_user = address
    email_password = "ezegyjelszo"
    attachment_dir = "/Users/vigadam/Documents/Munka/Municip_email_project/attachments"

    # filter for account
    sent_in_account = wave_2.loc[wave_2["email_from"] == address]
    index = sent_in_account.index

    # log into account, select inbox
    mail = imaplib.IMAP4_SSL("imap.gmail.com", 993)
    mail.login(email_user, email_password)
    mail.select("Inbox")

    # get number of inbox mails
    resp, data = mail.search(None, "(UNSEEN)")
    items = data[0].decode().split()
    print(address, len(items))

    for item in items:
        # get raw email data
        resp, data = mail.fetch(item, "(RFC822)")
        raw = email.message_from_bytes(data[0][1])

        # decode email body
        try:
            msg_body = get_body(raw).decode("utf-8")
        except:
            msg_body = get_body(raw).decode("ISO-8859-1")

        # get response sender
        if type(raw["From"]) == email.header.Header:
            sender = "Passz"
        else:
            sender = raw["From"]

        # try to match response to delivery
        for i in index:
            if (
                sent_in_account.loc[i, "town"] in msg_body
                or sent_in_account.loc[i, "email_to"] == sender
                or sent_in_account.loc[i, "email_to"] in msg_body
                or sent_in_account.loc[i, "email_to"] == sender.split()[-1][1:-1]
                or (
                    wave_2.loc[i, "email_to"].split("@")[1] in msg_body
                    and wave_2.loc[i, "email_to"].split("@")[1] not in hosts_cleaned
                )
                or (
                    wave_2.loc[i, "email_to"].split("@")[1] in sender
                    and wave_2.loc[i, "email_to"].split("@")[1] not in hosts_cleaned
                )
            ):

                found = True

                if type(wave_2.loc[i, "response"]) != str:

                    wave_2.loc[i, "response"] = msg_body
                    wave_2.loc[i, "resp_sender"] = sender

                    time_string = raw.values()[1].split("\n")[1].strip()[5:-15]
                    wave_2.loc[i, "resp_date"] = datetime.datetime.strptime(
                        time_string, "%d %b %Y %H:%M"
                    ) + datetime.timedelta(hours=9)
                    wave_2.loc[i, "resp_num"] = 1

                else:

                    answers = wave_2.loc[i, "response"]

                    if type(answers) == list:
                        answers.append(msg_body)
                        wave_2.loc[i, "response"] == answers
                    elif type(answers) == str:
                        answers = [answers]
                        answers.append(msg_body)
                        wave_2.loc[i, "response"] == answers

                    resp_senders = wave_2.loc[i, "resp_sender"]

                    if type(resp_senders) == list:
                        resp_senders.append(sender)
                        wave_2.loc[i, "resp_sender"] == resp_senders
                    elif type(answers) == str:
                        resp_senders = [resp_senders]
                        resp_senders.append(sender)
                        wave_2.loc[i, "resp_sender"] == resp_senders

                    wave_2.loc[i, "resp_num"] += 1

                break

            else:
                found = False

        # in case of no match, write to not_found df
        if found == False:
            not_found = not_found.append(
                {"sender": sender, "msg_body": msg_body, "account": address},
                ignore_index=True,
            )

        get_attachments(raw)

matenagy1983@gmail.com 0
lakatos.evelin1988@gmail.com 0
bernadett.toth1986@gmail.com 1
molnar.panna84@gmail.com 1
kolompar.mario84@gmail.com 2
kanalas.kevin86@gmail.com 1
petervarga0086@gmail.com 0
orsos.dzsenifer88@gmail.com 5


In [9]:
wave_2.to_excel("w2_answers_0827.xlsx", index=False)
wave_2.to_csv("w2_answers_0827.csv", index=False)

In [10]:
not_found.to_excel("not_found.xlsx")

In [11]:
not_found

,sender,msg_body,account


## response and bounce statistics

In [1]:
import pandas as pd

In [3]:
statsinput = pd.read_excel("w2_answers_0824.xlsx")#.drop(columns = "Unnamed: 0")

In [4]:
statsinput["bounced"] = None
for i in range(statsinput.shape[0]):
    if (
        statsinput.loc[i, "resp_sender"]
        == statsinput["resp_sender"].value_counts().index[0]
    ):
        statsinput.loc[i, "bounced"] = True

In [5]:
statsinput = statsinput.rename(
    columns={"response": "answer", "resp_date": "answ_time", "ping_sent": "ping"}
)

In [18]:
statsinput.to_csv("wave2_answ_0824.csv", index=False)

In [9]:
b-statsinput.loc[0,"answ_time"]

TypeError: unsupported operand type(s) for -: 'str' and 'Timestamp'

get time spend btw messages

In [16]:
import time
import datetime
import ciso8601

In [22]:
statsinput.loc[3,"answ_time"] - ciso8601.parse_datetime(statsinput.loc[3,"sent_time"])

Timedelta('1 days 20:10:41.719827')

In [2]:
import pandas as pd

In [3]:
statsinput = pd.read_excel("w2_answers_0828.xlsx")#.drop(columns="Unnamed: 0")

In [7]:
spam_name = "Mail Delivery Subsystem <mailer-daemon@googlemail.com>"

In [10]:
type_list = ["roma", "white", "tasz", "notasz"]

types = {
    "roma": statsinput.loc[statsinput["ethnicity"] == "Roma"],
    "white": statsinput.loc[statsinput["ethnicity"] == "White"],
    "tasz": statsinput.loc[statsinput["tasz"] == "treatment"],
    "notasz": statsinput.loc[statsinput["tasz"] == "control"],
}

type_stats = {}

for t in type_list:
    resp = list(types[t]["response"].dropna())

    bounced = types[t].loc[types[t]["resp_sender"] == spam_name].shape[0]

    bounce_and_resp = {
        "sent": types[t].shape[0],
        "bounced": bounced,
        "responded": len(resp) - bounced,
    }

    type_stats[t] = bounce_and_resp

pd.DataFrame.from_dict(type_stats)

,roma,white,tasz,notasz
sent,605,655,626,634
bounced,84,92,92,84
responded,258,383,317,324


In [11]:
days_list = [1, 2, 3, 4, 5, 6, 7]

day_stats = {}

for d in days_list:
    day = statsinput.loc[statsinput["day"] == d]
    resp = list(day["response"].dropna())

    bounced = day.loc[day["resp_sender"] == spam_name].shape[0]

    bounce_and_resp = {
        "sent": day.shape[0],
        "bounced": bounced,
        "responded": len(resp) - bounced,
    }

    day_stats[d] = bounce_and_resp

pd.DataFrame.from_dict(day_stats)

,1,2,3,4,5,6,7
sent,189,186,184,182,176,172,171
bounced,18,18,22,27,23,34,34
responded,126,110,103,94,81,69,58


In [14]:
wave_2

,index,town,municipality,muni_type,county.x,number_of_towns,wave,email_to,request,ethnicity,...,body,status,day,sent_time,response,resp_sender,resp_date,resp_num,ping,ping_body
0,1,Aba,Aba Nagyközség Önkormányzata,polgármesteri hivatal,Fejér,1,2,onkormanyzat@aba.hu,Wedding,Roma,...,Kedves Ügyintéző!\n\nPárommal esküvői helyszin...,Mail sent,1,2020-08-10 19:08:07.166295,Kedves Kevin!\r\nÖrömmel tölt el bennünket /bé...,NaN,2020-08-11 11:12:00,7.0,False,NaN
1,3,Abasár,Abasár Községi Önkormányzat,polgármesteri hivatal,Heves,1,2,abasar@abasar.hu,Nursery,Roma,...,Tisztelt Hölgyem/Uram!\n\nCsaládunk fontolgatj...,Mail sent,1,2020-08-10 19:08:10.960042,NaN,NaN,NaT,NaN,False,NaN
2,5,Sima,Abaújszántói,közös önkormányzati hivatal,Borsod-Abaúj-Zemplén,5,2,titkar@abaujszanto.hu,Camping,White,...,Tisztelt Önkormányzat!\n\nCsaládommal egy keré...,Mail sent,1,2020-08-10 19:08:14.952017,Tisztelt Varga Péter!\r\n\r\nSima községének k...,NaN,2020-08-11 07:46:00,2.0,True,"Kedves Ügyintéző,\n\nMikor kapok választ a kér..."
3,7,Börcs,Abdai,közös önkormányzati hivatal,Győr-Moson-Sopron,3,2,polgarmester@borcs.hu,Wedding,Roma,...,Kedves Ügyintéző!\n\nPárommal esküvői helyszin...,Mail sent,1,2020-08-10 19:08:18.280173,Kedves Kevin!\r\n\r\nSzívesen segítünk terveik...,<igazgatas@borcs.hu>,2020-08-12 15:19:00,4.0,False,NaN
4,9,Abony,Abony Város Önkormányzata,polgármesteri hivatal,Pest,1,2,abony@abony.hu,Cemetary,White,...,Tisztelt Ügyintéző!\n\nCsaládommal szeretnénk ...,Mail sent,1,2020-08-10 19:08:21.608649,Tisztelt Varga Péter!\r\n\r\nLevelem melléklet...,NaN,2020-08-12 11:21:00,2.0,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,2511,Őrbottyán,Őrbottyán Város Önkormányzata,polgármesteri hivatal,Pest,1,2,orbottyan@orbottyan.hu,Wedding,White,...,Kedves Ügyintéző!\n\nPárommal esküvői helyszin...,Mail sent,7,2020-08-19 10:42:06.371228,Tisztelt Ügyfelünk!\r\n\r\nKöszönjük megkeresé...,<orbottyan@orbottyan.hu>,2020-08-19 10:42:00,2.0,True,"Kedves Ügyintéző,\n\nMikor kapok választ a kér..."
1256,2513,Őrhalom,Őrhalmi,közös önkormányzati hivatal,Nógrád,4,2,hiv2671@freemail.hu,Cemetary,Roma,...,Tisztelt Ügyintéző!\n\nCsaládommal szeretnénk ...,Mail sent,7,2020-08-19 10:42:10.410927,\r\n** Üzenet letiltva **\r\n\r\nA(z) hiv2671@...,Mail Delivery Subsystem <mailer-daemon@googlem...,2020-08-19 10:49:00,1.0,True,Tisztelt Ügyintéző!\n\nKérhetek tájékoztatást ...
1257,2515,Őr,Őri,közös önkormányzati hivatal,Szabolcs-Szatmár-Bereg,2,2,polghivor@gmail.com,Nursery,Roma,...,Tisztelt Hölgyem/Uram!\n\nCsaládunk fontolgatj...,Mail sent,7,2020-08-19 10:42:14.214705,"<div dir=""ltr""><div> </div><div> Köszönjük le...","""=?UTF-8?Q?=C5=90r_Polg=C3=A1rmesteri_Hivatal?...",2020-08-19 10:42:00,1.0,True,Tisztelt Ügyintéző!\n\nKérhetek tájékoztatást ...
1258,2517,Szatta,Őriszentpéteri,közös önkormányzati hivatal,Vas,13,2,polgarmester.szatta@gmail.com,Camping,White,...,Tisztelt Önkormányzat!\n\nCsaládommal egy keré...,Mail sent,7,2020-08-19 10:42:17.699420,NaN,NaN,NaT,NaN,False,NaN


pings correct

In [60]:
wave_2 = pd.read_excel("w2_answers_0828.xlsx")#.drop(columns=["Unnamed: 0"])
wave_2 = wave_2.drop(columns=["ping","ping_body"])

In [26]:
ping1 = pd.read_csv("ping_d1.csv").drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
ping2 = pd.read_csv("ping_d2.csv").drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
ping3 = pd.read_csv("ping_d3.csv")#.drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
ping4 = pd.read_csv("ping_d4.csv")#.drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
ping5 = pd.read_csv("ping_d5.csv")#.drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
ping6 = pd.read_csv("ping_d6.csv")#.drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
ping7 = pd.read_csv("ping_d7.csv")#.drop(columns=["Unnamed: 0", "Unnamed: 0.1"])

In [41]:
ping = pd.concat([ping1, ping2, ping3, ping4, ping5, ping6, ping7,]).reset_index(
    drop=True
)

In [61]:
wave_2["ping"] = False
for i in range(ping.shape[0]):
    index = wave_2.loc[wave_2["index"] == ping.loc[i, "index"]].index
    wave_2.loc[index, "ping"] = True

In [62]:
wave_2["ping"].value_counts()

False    976
True     284
Name: ping, dtype: int64

#### bounce corr

In [ ]:
spam_name = "Mail Delivery Subsystem <mailer-daemon@googlemail.com>"

In [64]:
wave_2["bounced"] = False
for i in range(wave_2.shape[0]):
    if wave_2.loc[i, "resp_sender"] == spam_name:
        wave_2.loc[i, "bounced"] = True

In [65]:
wave_2["bounced"].value_counts()

False    1084
True      176
Name: bounced, dtype: int64

In [66]:
wave_2

,index,town,municipality,muni_type,county.x,number_of_towns,wave,email_to,request,ethnicity,...,body,status,day,sent_time,response,resp_sender,resp_date,resp_num,ping,bounced
0,1,Aba,Aba Nagyközség Önkormányzata,polgármesteri hivatal,Fejér,1,2,onkormanyzat@aba.hu,Wedding,Roma,...,Kedves Ügyintéző!\n\nPárommal esküvői helyszin...,Mail sent,1,2020-08-10 19:08:07.166295,Kedves Kevin!\r\nÖrömmel tölt el bennünket /bé...,NaN,2020-08-11 11:12:00,7.0,False,False
1,3,Abasár,Abasár Községi Önkormányzat,polgármesteri hivatal,Heves,1,2,abasar@abasar.hu,Nursery,Roma,...,Tisztelt Hölgyem/Uram!\n\nCsaládunk fontolgatj...,Mail sent,1,2020-08-10 19:08:10.960042,NaN,NaN,NaT,NaN,False,False
2,5,Sima,Abaújszántói,közös önkormányzati hivatal,Borsod-Abaúj-Zemplén,5,2,titkar@abaujszanto.hu,Camping,White,...,Tisztelt Önkormányzat!\n\nCsaládommal egy keré...,Mail sent,1,2020-08-10 19:08:14.952017,Tisztelt Varga Péter!\r\n\r\nSima községének k...,NaN,2020-08-11 07:46:00,2.0,False,False
3,7,Börcs,Abdai,közös önkormányzati hivatal,Győr-Moson-Sopron,3,2,polgarmester@borcs.hu,Wedding,Roma,...,Kedves Ügyintéző!\n\nPárommal esküvői helyszin...,Mail sent,1,2020-08-10 19:08:18.280173,Kedves Kevin!\r\n\r\nSzívesen segítünk terveik...,<igazgatas@borcs.hu>,2020-08-12 15:19:00,4.0,False,False
4,9,Abony,Abony Város Önkormányzata,polgármesteri hivatal,Pest,1,2,abony@abony.hu,Cemetary,White,...,Tisztelt Ügyintéző!\n\nCsaládommal szeretnénk ...,Mail sent,1,2020-08-10 19:08:21.608649,Tisztelt Varga Péter!\r\n\r\nLevelem melléklet...,NaN,2020-08-12 11:21:00,2.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,2511,Őrbottyán,Őrbottyán Város Önkormányzata,polgármesteri hivatal,Pest,1,2,orbottyan@orbottyan.hu,Wedding,White,...,Kedves Ügyintéző!\n\nPárommal esküvői helyszin...,Mail sent,7,2020-08-19 10:42:06.371228,Tisztelt Ügyfelünk!\r\n\r\nKöszönjük megkeresé...,<orbottyan@orbottyan.hu>,2020-08-19 10:42:00,2.0,False,False
1256,2513,Őrhalom,Őrhalmi,közös önkormányzati hivatal,Nógrád,4,2,hiv2671@freemail.hu,Cemetary,Roma,...,Tisztelt Ügyintéző!\n\nCsaládommal szeretnénk ...,Mail sent,7,2020-08-19 10:42:10.410927,\r\n** Üzenet letiltva **\r\n\r\nA(z) hiv2671@...,Mail Delivery Subsystem <mailer-daemon@googlem...,2020-08-19 10:49:00,1.0,False,True
1257,2515,Őr,Őri,közös önkormányzati hivatal,Szabolcs-Szatmár-Bereg,2,2,polghivor@gmail.com,Nursery,Roma,...,Tisztelt Hölgyem/Uram!\n\nCsaládunk fontolgatj...,Mail sent,7,2020-08-19 10:42:14.214705,"<div dir=""ltr""><div> </div><div> Köszönjük le...","""=?UTF-8?Q?=C5=90r_Polg=C3=A1rmesteri_Hivatal?...",2020-08-19 10:42:00,1.0,False,False
1258,2517,Szatta,Őriszentpéteri,közös önkormányzati hivatal,Vas,13,2,polgarmester.szatta@gmail.com,Camping,White,...,Tisztelt Önkormányzat!\n\nCsaládommal egy keré...,Mail sent,7,2020-08-19 10:42:17.699420,NaN,NaN,NaT,NaN,False,False


In [67]:
wave_2.to_excel("w2_answer_ping_0828.xlsx", index=False)